In [1]:
import re
import numpy as np

In [37]:
def read_input(filename):
    clay = []
    pattern = re.compile(r'(y|x)=(\d+), (x|y)=(\d+)\.\.(\d+)')
    with open(filename, 'r') as f:
        for line in f.readlines():
            m = pattern.search(line.strip())
            c1 = m[1]
            v1 = int(m[2])
            v2 = int(m[4])
            v3 = int(m[5])
            if c1 == 'x':
                for i in range(v2, v3 + 1):
                    clay.append((i, v1))
            elif c1 == 'y':
                for j in range(v2, v3 + 1):
                    clay.append((v1, j))
    i_min = min(clay, key=lambda z: z[0])[0]
    i_max = max(clay, key=lambda z: z[0])[0]
    j_min = min(clay, key=lambda z: z[1])[1]
    j_max = max(clay, key=lambda z: z[1])[1]
    grid = np.full((i_max + 1 + 1, j_max - j_min + 3 + 1), 1)
    spring = (0, 500 - j_min + 1 + 1)
    grid[spring] = 0
    for c in clay:
        grid[c[0], c[1] - j_min + 1 + 1] = 4
    
    return grid, spring

def print_grid(grid):
    for i in range(grid.shape[0]):
        line = []
        for v in grid[i, :]:
            if v == 0:
                c = '+'
            elif v == 1:
                c = '.'
            elif v == 2:
                c = '|'
            elif v == 3:
                c = '~'
            elif v == 4:
                c = '#'
            line.append(c)
        print(''.join(line))

def step(grid, springs):
    def step_spring(spring):
        i, j = spring
        d = np.argmax(grid[i + 1:, j] > 2)
        if not (d > 0 or grid[i + 1, j] > 2):
            d = grid.shape[0] - 1
        grid[i + 1:i + d, j] = 2
        if d + 1 < grid.shape[0]:
            l = np.argmax(np.concatenate((grid[i + d, :j][::-1] > 2, [True])))
            r = np.argmax(np.concatenate((grid[i + d, j + 1:] > 2, [True])))
            b = np.all(grid[i + d + 1, j - l:j + r + 1] >= 2)
            if b:
                grid[i + d, j] = 3
                if l > 0:
                    grid[i + d, j - l:j] = 3
                if r > 0:
                    grid[i + d, j + 1:j + r + 1] = 3
            else:
                l = min(l, np.argmin(grid[i + d + 1, :j + 1][::-1] > 2))
                r = min(r, np.argmin(grid[i + d + 1, j:] > 2))
                grid[i + d, j - l:j + r + 1] = 2
                if grid[i + d + 1, j - l] == 1:
                    springs.append((i + d, j - l))
                if grid[i + d + 1, j + r] == 1:
                    springs.append((i + d, j + r))
    
    for spring in springs:
        step_spring(spring)
    return springs
    

def run_flow(grid, spring):
    springs = [spring]
    pre = np.zeros_like(grid)
    i = 0
    while not np.array_equal(grid, pre):
        i += 1
        pre = np.copy(grid)
        springs = step(grid, springs)
    dry_cells = np.count_nonzero(grid[7:1985, :] == 2)
    water_cells = np.count_nonzero(grid[7:1985, :] == 3)
    return dry_cells + water_cells, water_cells

In [39]:
grid, spring = read_input('17_input.txt')
run_flow(grid, spring)

(29741, 24198)